In [1]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
import pytesseract
from langchain_community.document_loaders.parsers import TesseractBlobParser
import time
from statistics import mean

# Tekstextractie uit PDF-bestanden - Tesktformaat

## 1. Doelstelling

De meeste informatiebronnen waarover we beschikken zijn in PDF-formaat, of kunnen eenvoudig naar PDF worden omgezet. Aangezien een RAG-pijplijn werkt met tekst en niet met bestanden, is het belangrijk dat we de inhoud van deze documenten kunnen extraheren.

De meest voor de hand liggende manier om dit aan te pakken, is door simpelweg de platte tekst uit de PDF-bestanden te halen. Er bestaan verschillende tools die dit mogelijk maken, waarvan de bekendste waarschijnlijk [PyMuPDF](https://pymupdf.readthedocs.io/en/latest/) is.

We zullen dit tool, via de [Langchain wrapper](https://python.langchain.com/docs/integrations/document_loaders/pymupdf/), gebruiken om het volgende na te gaan:

- Hoe wordt het gebruikt? Is het gebruiksvriendelijk?
- Hoeveel tijd neemt de omzetting in beslag?
- Is de gegenereerde tekst van goede kwaliteit?

## 2. Methodologie

Om de bruikbaarheid en efficiëntie — met andere woorden, de kwaliteit van de gegenereerde tekst en de tijd die de omzetting in beslag neemt — van PyMuPDF te evalueren, zullen we vier types PDF-bestanden omzetten. Bij elke omzetting meten we de benodigde tijd met behulp van de `time`-module en beoordelen we in welke mate de gegenereerde tekst trouw blijft aan de oorspronkelijke inhoud van de PDF.

De vier bestandssoorten die we testen zijn:

- **Studiewijzers**: Deze zijn beschikbaar via Chamilo en maken dus deel uit van een webpagina. Chamilo biedt de mogelijkheid om studiewijzers te exporteren naar PDF-formaat.
- **Slides**: Dit zijn PDF-bestanden die rechtstreeks door docenten worden aangeleverd en de leerstof van een vak bevatten. Er is hierbij geen verdere verwerking nodig.
- **Bijzondere slides**: Voor het vak *Infrastructure Automation* (en ook voor AI & Data Science) zijn de slides oorspronkelijk geschreven in Markdown, omgezet via Pandoc naar presentaties en gepubliceerd via GitHub Pages. De originele Markdown-bestanden zijn beschikbaar op GitHub. In dit geval willen we nagaan of de tools een resultaat kunnen genereren dat vergelijkbaar is met de oorspronkelijke Markdown.
- **datalinux.pdf**: Dit is de syllabus van het vak Linux for Data Scientists. Het is een omvangrijk document van ongeveer 300 pagina’s. Het doel hier is om de tools tot het uiterste te testen en te observeren hoeveel tijd ze nodig hebben om een dergelijk groot bestand te verwerken.

## 3. Uittesten

In [2]:
# Definiêren van de verschillende paden waarin de  bestanden opgeslagen zijn/zullen worden.
pdfs_path = './pdfs/'
texts_path = './texts/'

studiewijzers_path = 'Studiewijzers/'
slides_path = 'Slides/'
special_slides_path = 'Bijzondere_slides/'
syllabus_path = 'Syllabus/'

In [3]:
#pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [4]:
# This helpfunction will measure the times it takes to convert a file
def chrono(path_to_pdfs, path_to_texts):
    time_taken = {}
    times = []

    for e in os.scandir(path_to_pdfs):
        start = time.time()
        
        filename = e.name[:-4]
        loader = PyMuPDFLoader(
            e.path,
            mode='single',
            extract_tables="markdown",
            #images_inner_format='text',
            #images_parser=TesseractBlobParser(),
        )
        doc = loader.load()
        text = doc[0].page_content
        
        with open(f'{os.path.join(path_to_texts, filename)}.txt', "w", encoding="utf-8") as f:
            f.write(text)
        
        end = time.time()
        
        time_taken[filename] = end - start

    for f, t in time_taken.items():
        print(f'{f}: {t}s')
        times.append(t)

    print(f'\nTotal time taken: {sum(times)} s')
    print(f'Average time taken: {mean(times)} s\n')

### 3.1. Studiewijzers

In [5]:
studiewijzers_pdfs_path = os.path.join(pdfs_path, studiewijzers_path)
studiewijzers_texts_path = os.path.join(texts_path, studiewijzers_path)

In [6]:
chrono(studiewijzers_pdfs_path, studiewijzers_texts_path)

Studiewijzer_AI_Data_Science: 10.243871688842773s
Studiewijzer_Infrastructure_Automation: 4.031424522399902s
Studiewijzer_linux_for_Data_Scientists: 5.372938871383667s

Total time taken: 19.648235082626343 s
Average time taken: 6.549411694208781 s



### 3.1. Slides

In [7]:
slides_pdfs_path = os.path.join(pdfs_path, slides_path)
slides_texts_path = os.path.join(texts_path, slides_path)

In [8]:
chrono(slides_pdfs_path, slides_texts_path)

dsai-en-0-intro: 0.3008284568786621s
dsai-en-1-sampling: 0.28070974349975586s
dsai-en-2-univariate: 0.40171027183532715s
dsai-en-3a-central-limit-theorem: 1.3058478832244873s
dsai-en-3b-hypothesis-testing: 0.46152591705322266s
dsai-en-4-bivariate-qual-qual: 0.5171277523040771s
dsai-en-5-bivariate-qual-quant: 0.3396742343902588s
dsai-en-6-bivariate-quant-quant: 0.29468441009521484s
dsai-en-7-timeseries: 0.3831017017364502s

Total time taken: 4.285210371017456 s
Average time taken: 0.4761344856686062 s



### 3.3. Bijzondere slides

In [9]:
special_slides_pdfs_path = os.path.join(pdfs_path, special_slides_path)
special_slides_texts_path = os.path.join(texts_path, special_slides_path)

In [10]:
chrono(special_slides_pdfs_path, special_slides_texts_path)

1. Continuous Integration_ Deployment with Jenkins: 0.4373805522918701s
2. Configuration Management with Ansible: 0.49861931800842285s
3. Container orchestration with Kubernetes: 0.8254592418670654s
4. Monitoring with Prometheus: 0.45254087448120117s
Infrastructure Automation_ inleiding: 0.5080986022949219s

Total time taken: 2.7220985889434814 s
Average time taken: 0.5444197177886962 s



### 3.4. Syllabus

In [11]:
syllabus_pdfs_path = os.path.join(pdfs_path, syllabus_path)
syllabus_texts_path = os.path.join(texts_path, syllabus_path)

In [12]:
chrono(syllabus_pdfs_path, syllabus_texts_path)

datalinux: 18.77149248123169s

Total time taken: 18.77149248123169 s
Average time taken: 18.77149248123169 s



## 4. Besluit

> **Opgelet**: De cijfers die hier besproken worden, zijn geldig op 27 april 2025. We kunnen hun nauwkeurigheid na deze datum niet garanderen.
> 
Het opzetten en gebruiken van PyMuPDF is zeer eenvoudig; het draait om het correct instantieren van de juiste objecten en het aanroepen van de juiste methoden. Bovendien zijn de verwerkingstijden snel:

|                   | Gemiddele verwerkingstijd   |
|-------------------|-----------------------------|
| Studiewijzers     | 7s                          |
| Slides            | 0.5s                        |
| Bijzondere slides | 0.5s                        |
| Syllabus          | 19s                         |

De gegenereerde tekst is va, goede kwaliteit; op basis van onze waarnemingen lijkt het dat de gehele inhoud van de PDF-bestanden succesvol wordt geëxtraheerd. Dit pluspunt is echter ook een minpunt, omdat PyMuPDF alles extraheert, inclusief ruis in de data (zoals paginanummering op slides).

Het probleem bij het extraheren van platte tekst is dat we de structuur van het PDF-bestand verliezen, terwijl achter die structuur semantische informatie schuilgaat. We geloven dat dit het chunkingproces ingewikkelder zal maken, omdat er meer moeite gestoken moet worden in het creëren van samenhangende en betekenisvolle chunks die aan de LLM gevoed zullen worden.